# core

> Core functions

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import numpy as np
import pandas as pd
from tqdm import tqdm
from typing import List

In [ ]:
#| export
def add_missing_slots(
        df: pd.DataFrame,    # input dataframe with datetime, entity and value columns - time series format
        datetime_col: str,   # name of the datetime column
        entity_col: str,     # name of the entity column. If a time series is associated to a location, this column will be 'location_id'
        value_col: str,      # name of the value column
        freq: str='H',       # frequency of the time series. Default is hourly
        fill_value: int = 0  # value to use to fill missing slots
) -> pd.DataFrame:
    """
    Add missing slots to a time series dataframe.
    This function is useful to fill missing slots in a time series dataframe.
    For example, if a time series is associated to a location, this function will add missing slots for each location.
    Missing slots are filled with the value specified in the 'fill_value' parameter.
    By default, the frequency of the time series is hourly.
    """

    entity_ids = df[entity_col].unique()
    all_hours = pd.date_range(start=df[datetime_col].min(), end=df[datetime_col].max(), freq=freq)

    output = pd.DataFrame()

    for entity_id in tqdm(entity_ids):

        # keep only rides for this 'location_id'
        df_entity_id = df.loc[df[entity_col] == entity_id, [datetime_col, value_col]]

        # quick way to add missing dates with 0 in a Series
        # taken from https://stackoverflow.com/a/19324591
        df_entity_id.set_index(datetime_col, inplace=True)
        df_entity_id.index = pd.DatetimeIndex(df_entity_id.index)
        df_entity_id = df_entity_id.reindex(all_hours, fill_value=0)

        # add back 'location_id' column
        df_entity_id[entity_col] = entity_id

        output = pd.concat([output, df_entity_id])

    # move the purchase_day from index to column
    output = output.reset_index().rename(columns={'index': datetime_col})
    output = output[[datetime_col, entity_col, value_col]].copy()

    return output

In [ ]:
df = pd.DataFrame({
    'pickup_hour': ['2022-01-01 00:00:00', '2022-01-01 01:00:00', '2022-01-01 03:00:00', '2022-01-01 01:00:00', '2022-01-01 02:00:00', '2022-01-01 05:00:00'],
    'pickup_location_id': [1, 1, 1, 2, 2, 2],
    'rides': [2, 3, 1, 1, 2, 1]
})
df

,pickup_hour,pickup_location_id,rides
0,2022-01-01 00:00:00,1,2
1,2022-01-01 01:00:00,1,3
2,2022-01-01 03:00:00,1,1
3,2022-01-01 01:00:00,2,1
4,2022-01-01 02:00:00,2,2
5,2022-01-01 05:00:00,2,1


In [ ]:
add_missing_slots(df, datetime_col='pickup_hour', entity_col='pickup_location_id', value_col='rides', freq='H')

100%|██████████| 2/2 [00:00<00:00, 448.83it/s]


,pickup_hour,pickup_location_id,rides
0,2022-01-01 00:00:00,1,2
1,2022-01-01 01:00:00,1,3
2,2022-01-01 02:00:00,1,0
3,2022-01-01 03:00:00,1,1
4,2022-01-01 04:00:00,1,0
5,2022-01-01 05:00:00,1,0
6,2022-01-01 00:00:00,2,0
7,2022-01-01 01:00:00,2,1
8,2022-01-01 02:00:00,2,2
9,2022-01-01 03:00:00,2,0


In [ ]:
#| export
def get_cutoff_indices_features_and_target(
        ts_data: pd.DataFrame,     # Time Series DataFrame
        datetime_col: str,         # Name of the datetime column
        n_features: int,           # Number of features to use for the prediction
        n_targets: int=1,           # Number of target values to predict
        step_size: int=1,          # Step size to use to slide the Time Series DataFrame
) -> List[tuple]:
    """Function to get the indices for the cutoffs of a Time Series DataFrame.
       The Time Series DataFrame should be orderded by time."""

    # check if it is ordered
    if not ts_data[datetime_col].is_monotonic_increasing:
        raise ValueError("The DataFrame should be ordered by time.")
    
    # check if the index is ordered
    if not ts_data.index.is_monotonic_increasing:
        raise ValueError("The index should be ordered by time.")
    
    # check if the step_size is valid
    if step_size < 1:
        raise ValueError("The step_size should be >= 1.")
    
    # check if the number of features is valid
    if n_features < 1:
        raise ValueError("The number of features should be >= 1.")
    
    # check if the number of target values is valid
    if n_targets < 1:
        raise ValueError("The number of target values should be >= 1.")
    
    # check if the number of features and target values is valid
    if n_features + n_targets > len(ts_data):
        raise ValueError("The number of features + target values should be <= the length of the time series.")
    
    # below we compute the indices for the cutoffs
    stop_position = len(ts_data) - 1

    # Start the first sub-sequence at index position 0
    subseq_first_idx = 0
    subseq_mid_idx = n_features
    subseq_last_idx = n_features + n_targets
    indices = []

    while subseq_last_idx <= stop_position:
        indices.append((subseq_first_idx, subseq_mid_idx, subseq_last_idx))

        subseq_first_idx += step_size
        subseq_mid_idx += step_size
        subseq_last_idx += step_size

    return indices

In [ ]:
# build a time series dataframe with 10 hours of data in random order
ts_data = pd.DataFrame({
    'pickup_hour': ['2022-01-01 01:00:00', '2022-01-01 00:00:00', '2022-01-01 03:00:00', '2022-01-01 04:00:00', '2022-01-01 02:00:00', '2022-01-01 05:00:00', '2022-01-01 09:00:00', '2022-01-01 06:00:00', '2022-01-01 07:00:00', '2022-01-01 08:00:00'],
    'rides': [2, 3, 1, 1, 2, 1, 1, 2, 1, 1]
})
ts_data

,pickup_hour,rides
0,2022-01-01 01:00:00,2
1,2022-01-01 00:00:00,3
2,2022-01-01 03:00:00,1
3,2022-01-01 04:00:00,1
4,2022-01-01 02:00:00,2
5,2022-01-01 05:00:00,1
6,2022-01-01 09:00:00,1
7,2022-01-01 06:00:00,2
8,2022-01-01 07:00:00,1
9,2022-01-01 08:00:00,1


In [ ]:
# the time series should be ordered by time, otherwise it will not work and throw a ValueError
ts_data.sort_values(by='pickup_hour', inplace=True, ignore_index=True)
cutoff_idxs = get_cutoff_indices_features_and_target(ts_data, datetime_col='pickup_hour', n_features=3, n_targets=2, step_size=1)
cutoff_idxs

[(0, 3, 5), (1, 4, 6), (2, 5, 7), (3, 6, 8), (4, 7, 9)]

In [ ]:
assert cutoff_idxs == [(0, 3, 5), (1, 4, 6), (2, 5, 7), (3, 6, 8), (4, 7, 9)]

In [ ]:
#| export
def transform_ts_data_into_features_and_target(
    ts_data: pd.DataFrame,   # Time Series DataFrame
    n_features: int,         # Number of features to use for the prediction
    datetime_col: str,       # Name of the datetime column
    entity_col: str,         # Name of the entity column, e.g. location_id
    value_col: str,          # Name of the value column
    n_targets: int=1,        # Number of target values to predict
    step_size: int=1,        # Step size to use to slide the Time Series DataFrame
    step_name: str=None,     # Name of the step column
    concat_Xy: bool=False    # Whether to concat X and y on the same dataframe or not
) -> pd.DataFrame:
    """
    Slices and transposes data from time-series format into a (features, target)
    format that we can use to train Supervised ML models.
    """

    entity_ids = ts_data[entity_col].unique()
    features = pd.DataFrame()
    targets = pd.DataFrame()
    
    for entity_id in tqdm(entity_ids):
        
        # keep only ts data for this `location_id`
        ts_data_one_location = ts_data.loc[
            ts_data[entity_col] == entity_id, 
            [datetime_col, value_col]
        ]

        # pre-compute cutoff indices to split dataframe rows
        indices = get_cutoff_indices_features_and_target(
            ts_data=ts_data_one_location,
            datetime_col=datetime_col,
            n_features=n_features,
            n_targets=n_targets,
            step_size=step_size
        )

        # slice and transpose data into numpy arrays for features and targets
        time_values = []
        n_examples = len(indices)
        x = np.ndarray(shape=(n_examples, n_features), dtype=np.float32)
        if n_targets == 1:
            y = np.ndarray(shape=(n_examples), dtype=np.float32)
            for i, idx in enumerate(indices):
                x[i, :] = ts_data_one_location.iloc[idx[0]:idx[1]][value_col].values
                y[i] = ts_data_one_location.iloc[idx[1]:idx[2]][value_col].values
                time_values.append(ts_data_one_location.iloc[idx[1]][datetime_col])
        else:
            y = np.ndarray(shape=(n_examples, n_targets), dtype=np.float32)
            for i, idx in enumerate(indices):
                x[i, :] = ts_data_one_location.iloc[idx[0]:idx[1]][value_col].values
                y[i, :] = ts_data_one_location.iloc[idx[1]:idx[2]][value_col].values
                time_values.append(ts_data_one_location.iloc[idx[1]][datetime_col])

        # numpy -> pandas
        if step_name is None:
            features_one_location = pd.DataFrame(
                x,
                columns=[f'{value_col}_previous_{i+1}' for i in reversed(range(n_features))]
            )
            features_one_location[datetime_col] = time_values
            features_one_location[entity_col] = entity_id

            # numpy -> pandas
            if n_targets == 1:
                targets_one_location = pd.DataFrame(y, columns=[f'target_{value_col}_next'])
            else:
                targets_one_location = pd.DataFrame(
                    y,
                    columns=[f'target_{value_col}_next_{i+1}' for i in range(n_targets)]
                )
        else:
            features_one_location = pd.DataFrame(
                x,
                columns=[f'{value_col}_previous_{i+1}_{step_name}' for i in reversed(range(n_features))]
            )
            features_one_location[datetime_col] = time_values
            features_one_location[entity_col] = entity_id

            # numpy -> pandas
            if n_targets == 1:
                targets_one_location = pd.DataFrame(y, columns=[f'target_{value_col}_next_{step_name}'])
            else:
                targets_one_location = pd.DataFrame(
                    y,
                    columns=[f'target_{value_col}_next_{i+1}_{step_name}' for i in range(n_targets)]
                )
        
        # concatenate results
        features = pd.concat([features, features_one_location])
        targets = pd.concat([targets, targets_one_location])

    features.reset_index(inplace=True, drop=True)
    targets.reset_index(inplace=True, drop=True)

    if concat_Xy:
        return pd.concat([features, targets], axis=1)
    else:
        return features, targets

In [ ]:
# build a time series dataframe with 10 hours of data in random order and a location id column with 1 and 2
ts_data = pd.DataFrame({
    'pickup_hour': ['2022-01-01 01:00:00', '2022-01-01 00:00:00', '2022-01-01 03:00:00', '2022-01-01 04:00:00', '2022-01-01 02:00:00', '2022-01-01 05:00:00', '2022-01-01 09:00:00', '2022-01-01 06:00:00', '2022-01-01 07:00:00', '2022-01-01 08:00:00'],
    'location_id': [1, 1, 1, 1, 1, 1, 2, 2, 2, 2],
    'rides': [2, 3, 1, 1, 2, 1, 1, 2, 1, 1]
})
ts_data

,pickup_hour,location_id,rides
0,2022-01-01 01:00:00,1,2
1,2022-01-01 00:00:00,1,3
2,2022-01-01 03:00:00,1,1
3,2022-01-01 04:00:00,1,1
4,2022-01-01 02:00:00,1,2
5,2022-01-01 05:00:00,1,1
6,2022-01-01 09:00:00,2,1
7,2022-01-01 06:00:00,2,2
8,2022-01-01 07:00:00,2,1
9,2022-01-01 08:00:00,2,1


In [ ]:
ts_data = add_missing_slots(ts_data, datetime_col='pickup_hour', entity_col='location_id', value_col='rides', freq='1H')
ts_data

100%|██████████| 2/2 [00:00<00:00, 708.92it/s]


,pickup_hour,location_id,rides
0,2022-01-01 00:00:00,1,3
1,2022-01-01 01:00:00,1,2
2,2022-01-01 02:00:00,1,2
3,2022-01-01 03:00:00,1,1
4,2022-01-01 04:00:00,1,1
5,2022-01-01 05:00:00,1,1
6,2022-01-01 06:00:00,1,0
7,2022-01-01 07:00:00,1,0
8,2022-01-01 08:00:00,1,0
9,2022-01-01 09:00:00,1,0


In [ ]:
features, targets = transform_ts_data_into_features_and_target(
    ts_data=ts_data,
    n_features=3,
    datetime_col='pickup_hour',
    entity_col='location_id',
    value_col='rides',
    n_targets=2,
    step_size=1
)

100%|██████████| 2/2 [00:00<00:00, 371.60it/s]


In [ ]:
features

,rides_previous_3,rides_previous_2,rides_previous_1,pickup_hour,location_id
0,3.0,2.0,2.0,2022-01-01 03:00:00,1
1,2.0,2.0,1.0,2022-01-01 04:00:00,1
2,2.0,1.0,1.0,2022-01-01 05:00:00,1
3,1.0,1.0,1.0,2022-01-01 06:00:00,1
4,1.0,1.0,0.0,2022-01-01 07:00:00,1
5,0.0,0.0,0.0,2022-01-01 03:00:00,2
6,0.0,0.0,0.0,2022-01-01 04:00:00,2
7,0.0,0.0,0.0,2022-01-01 05:00:00,2
8,0.0,0.0,0.0,2022-01-01 06:00:00,2
9,0.0,0.0,2.0,2022-01-01 07:00:00,2


In [ ]:
targets

,target_rides_next_1,target_rides_next_2
0,1.0,1.0
1,1.0,1.0
2,1.0,0.0
3,0.0,0.0
4,0.0,0.0
5,0.0,0.0
6,0.0,0.0
7,0.0,2.0
8,2.0,1.0
9,1.0,1.0


In [ ]:
pd.concat([features, targets], axis=1)

,rides_previous_3,rides_previous_2,rides_previous_1,pickup_hour,location_id,target_rides_next_1,target_rides_next_2
0,3.0,2.0,2.0,2022-01-01 03:00:00,1,1.0,1.0
1,2.0,2.0,1.0,2022-01-01 04:00:00,1,1.0,1.0
2,2.0,1.0,1.0,2022-01-01 05:00:00,1,1.0,0.0
3,1.0,1.0,1.0,2022-01-01 06:00:00,1,0.0,0.0
4,1.0,1.0,0.0,2022-01-01 07:00:00,1,0.0,0.0
5,0.0,0.0,0.0,2022-01-01 03:00:00,2,0.0,0.0
6,0.0,0.0,0.0,2022-01-01 04:00:00,2,0.0,0.0
7,0.0,0.0,0.0,2022-01-01 05:00:00,2,0.0,2.0
8,0.0,0.0,0.0,2022-01-01 06:00:00,2,2.0,1.0
9,0.0,0.0,2.0,2022-01-01 07:00:00,2,1.0,1.0


This dataset could be use to predict the rides for the next 2 hours, for each location_id, by using historical rides from the previous 3 hours.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()